In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

     |████████████████████████████████| 2.0MB 6.0MB/s 
     |████████████████████████████████| 3.2MB 17.8MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e0c1fa26e64ab559ecb12abc978e23cd9f8f14a9bdb12aeaf0ba5af30a8f5bd8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import sys
# sys.path.remove('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions')

In [ ]:
for p in sys.path:
  print(p)


/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
/content/drive/MyDrive/Colab Notebooks/W266 Final Project/master/helper_functions


# Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import math
import random
import pickle
import time

from numpy.linalg import norm
from scipy.stats import rankdata
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten

import parse
import bert_funcs as bf
import evaluation

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/raw_data_preprocessed.csv')
df.head(5)

,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...","When I run my socket program, there get: `impo...",#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...","with socket.socket(socket.AF_INET, socket.SOCK...",#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1
2,55661929,How to display 5 numbers per line from a list?,<p>How to display 5 numbers per line from a li...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,How to display 5 numbers per line from a list?...,"lx = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,...",55662134,<p>There can be different approaches to the pr...,There can be different approaches to the probl...,There can be different approaches to the probl...,x=[] #your given list\nfor t in range(len(x))...,4,125,0,0
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,55662821,<p>Define list as <code>[]</code> and use <cod...,Define list as [] and use append to add elemen...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',...",2,24,0,1
4,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,create list with spectial type - Spyder error ...,>script[0][4][2]name 'script' is not defined\n...,55662828,<p>I modified your code slightly:</p>\n\n<pre>...,I modified your code slightly:\nscript = [['A'...,I modified your code slightly:\n\nand it works...,"script = [['A','A',['A'],['A'],['A','5','>'],'...",2,24,0,0


In [ ]:
df_bert = df[['question_id', 'question_title_body', 'answer_id', 'answer_body', 'is_accepted_answer']]
df_bert.head(5)

,question_id,question_title_body,answer_id,answer_body,is_accepted_answer
0,55661532,"When I run my socket program, there get: `impo...",55661630,The answer to your new question is that you ca...,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",55661660,"You can't use socket.socket(socket.AF_INET, so...",1
2,55661929,How to display 5 numbers per line from a list?...,55662134,There can be different approaches to the probl...,0
3,55662778,create list with spectial type - Spyder error ...,55662821,Define list as [] and use append to add elemen...,1
4,55662778,create list with spectial type - Spyder error ...,55662828,I modified your code slightly:\nscript = [['A'...,0


# Preprocessing

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Create Tokens Dictionary

In [ ]:
tokens_dict = {}

In [ ]:
# One dict for storing every question and answer ID associated with (token IDs, mask IDs, and segment IDs)
for question_id in df_bert['question_id'].unique():
  tokens, masks, segments = bf.bert_encode_content(df_bert.loc[df_bert['question_id'] == question_id].question_title_body.values[0], tokenizer, 512)
  tokens_dict[question_id] = (tokens, masks, segments)

for answer_id in df_bert['answer_id']:
  tokens, masks, segments = bf.bert_encode_content(df_bert.loc[df_bert['answer_id'] == answer_id].answer_body.values[0], tokenizer, 512)
  tokens_dict[answer_id] = (tokens, masks, segments)

In [ ]:
# Save tokens dictionary to file
tokens_dict_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/bert_tokens.pickle', 'wb')
pickle.dump(tokens_dict, tokens_dict_to_file)

In [ ]:
# Load tokens dictionary from file
tokens_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/bert_tokens.pickle', 'rb')
bert_tokens_dict = pickle.load(tokens_from_file)

### Create Embeddings Dictionary

In [ ]:
test_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_test.pickle', 'rb')
qa_dict_test = pickle.load(test_from_file)

In [ ]:
list(qa_dict_test.items())[0]

(55661929,
 [(55662134, 0),
  (55662053, 0),
  (55661981, 1),
  (64717925, 0),
  (52250747, 0),
  (47805233, 0),
  (60398311, 0),
  (47860893, 0),
  (50565510, 0),
  (54576739, 0)])

In [ ]:
embeddings_dict_test = {}

In [ ]:
len(embeddings_dict_test)

1339

In [ ]:
count = 0
for key, val in qa_dict_test.items():
  start = time.time()
  question_tokens = bert_tokens_dict[key]
  q_output = model(list(question_tokens))
  q_embedding = tf.reduce_mean(q_output.last_hidden_state[0], 0)
  embeddings_dict_test[key] = q_embedding

  for answer in val:
    answer_tokens = bert_tokens_dict[answer[0]]
    a_output = model(list(answer_tokens))
    a_embedding = tf.reduce_mean(a_output.last_hidden_state[0], 0)
    embeddings_dict_test[answer[0]] = a_embedding

  count += 1
  end = time.time()
  print(str(count) + ", Elapsed Time: " + str(end - start))

1, Elapsed Time: 27.395116806030273
2, Elapsed Time: 27.41688871383667
3, Elapsed Time: 27.256158590316772
4, Elapsed Time: 27.530065774917603
5, Elapsed Time: 27.33034610748291
6, Elapsed Time: 27.466731071472168
7, Elapsed Time: 27.342478036880493
8, Elapsed Time: 27.390989303588867
9, Elapsed Time: 27.36980891227722
10, Elapsed Time: 27.126490116119385
11, Elapsed Time: 28.157481908798218
12, Elapsed Time: 27.214085578918457
13, Elapsed Time: 27.461440801620483
14, Elapsed Time: 27.22043776512146
15, Elapsed Time: 27.568860292434692
16, Elapsed Time: 27.437520503997803
17, Elapsed Time: 27.668947458267212
18, Elapsed Time: 27.35059666633606
19, Elapsed Time: 27.376861333847046
20, Elapsed Time: 27.336936473846436
21, Elapsed Time: 27.290167808532715
22, Elapsed Time: 27.52889323234558
23, Elapsed Time: 27.195538759231567
24, Elapsed Time: 27.405508041381836
25, Elapsed Time: 27.288917541503906
26, Elapsed Time: 27.382194995880127
27, Elapsed Time: 27.171276569366455
28, Elapsed Time

KeyboardInterrupt: ignored

In [ ]:
# Loop through each question/answer => (token IDs, mask IDs, and segment IDs)
for key, val in bert_tokens_dict.items():
  # Input tokenizer output into Bert model
  output = model(list(val))
  # Compute sentence embedding by averaging last hidden states for each token
  embedding = tf.reduce_mean(output.last_hidden_state[0], 0)
  # Add sentence embedding to embeddings dict
  embeddings_dict[key] = embedding

KeyboardInterrupt: ignored

In [ ]:
# Save embeddings dictionary to file
embeddings_dict_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/bert_model_embeddings.pickle', 'wb')
pickle.dump(embeddings_dict, embeddings_dict_to_file)

In [ ]:
# Load embeddings dictionary from file
embeddings_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/embeddings/bert_model_embeddings.pickle', 'rb')
bert_embeddings_dict = pickle.load(embeddings_from_file)

In [ ]:
ei = tokenizer(df_bert['question_title_body'][0], return_tensors='tf', max_length=512, truncation=True)
model = TFBertModel.from_pretrained('bert-base-cased')
out = model(ei)
print(out.last_hidden_state)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


tf.Tensor(
[[[ 0.5129847   0.07739093 -0.35050282 ... -0.5410513   0.23043871
    0.29689854]
  [ 0.74136835 -0.6557424   0.66889775 ... -0.49493402  0.24674141
    0.1132701 ]
  [ 1.0126009  -0.10212907  0.55164886 ... -0.5098649   0.2549655
    0.25576   ]
  ...
  [ 0.33248731  0.6192787  -0.1220021  ...  0.30154145 -0.2293469
    0.39337248]
  [ 0.39108065 -0.09338865  0.13371217 ... -0.31179035  0.09566683
    0.53667814]
  [ 1.5757154   0.6163668   0.03409199 ... -0.48208538  0.33591938
    0.19467762]]], shape=(1, 512, 768), dtype=float32)


In [ ]:
out.last_hidden_state[0].shape

TensorShape([512, 768])

In [ ]:
np.array(tf.reduce_mean(out.last_hidden_state[0], 0)).shape

(768,)

In [ ]:
o = model(list(bert_tokens_dict[list(bert_tokens_dict.keys())[0]]))
print(o.last_hidden_state[0])

tf.Tensor(
[[ 0.47930267  0.1221507  -0.3594157  ... -0.4584473   0.42028385
   0.29491052]
 [ 0.9185776  -0.5071151   0.7459686  ... -0.38195616  0.49895388
   0.12480364]
 [ 0.59232306  0.91895807  0.1639419  ... -0.22477147  0.09215103
   0.08372055]
 ...
 [ 0.90595615  1.1666242  -0.51298517 ... -0.7121      0.45003214
   1.5841244 ]
 [ 0.82089794  1.0042609  -0.738871   ... -0.88243103  0.51292217
   1.6258527 ]
 [ 1.6149615   0.60641795  0.18744332 ... -0.5573716   0.331203
  -0.00794739]], shape=(512, 768), dtype=float32)
